 # PANDAS
  'Pandas' es un estándar de la industria para analizar datos en Python. Con unas pocas pulsaciones de teclas, puedes cargar, filtrar, reestructurar y visualizar gigabytes de información heterogénea. Está desarrollado sobre la biblioteca 'NumPy' y toma prestados muchos de sus conceptos y convenciones de sintaxis, por lo que si te sientes cómodo con 'NumPy', Pandas te resultará una herramienta bastante familiar. E incluso si nunca has oído hablar de 'NumPy', Pandas ofrece una gran oportunidad para resolver problemas de análisis de datos con poca o ninguna experiencia en programación.

Dos características clave que Pandas aporta a las matrices 'NumPy' son:

1. Tipos heterogéneos: cada columna puede tener su propio tipo;

2. Índice: mejora la velocidad de búsqueda de las columnas especificadas.

Resulta que estas características son suficientes para hacer de 'Pandas' un poderoso competidor tanto de las 'hojas de cálculo' como de las 'bases de datos'.

'Polars', la reciente reencarnación de Pandas (escrita en 'Rust', por lo tanto más rápida ¹) ya no usa 'NumPy' bajo el capó, pero la sintaxis es bastante similar, por lo que aprender 'Pandas' también te permitirá sentirte cómodo con 'Polars'.

<img src="Img/Articulo_2_diagram_1.png">

## Parte 1. Motivación y presentación

Supongamos que tienes un archivo con un millón de líneas de valores separados por comas como este:

<img src="Img/Articulo_2_diagram_2.png">

Y necesitas dar respuestas a preguntas básicas como "¿Qué ciudades tienen un área de más de 450 km² y una población de menos de 10 millones" con 'NumPy'.

La solución de fuerza bruta de introducir toda la tabla en una matriz, 'NumPy' no es una buena opción: normalmente, las matrices 'NumPy' son homogéneas (todos los valores tienen el mismo tipo), por lo que todos los campos se interpretarán como 'cadenas' y las comparaciones no funcionarán como se espera.

Sí, 'NumPy' tiene matrices estructuradas y de registros que permiten columnas de distintos tipos, pero están pensadas principalmente para interactuar con código 'C'. Cuando se utilizan para fines generales, tienen las siguientes desventajas:

- No es realmente intuitivo (por ejemplo, te enfrentarás a constantes como <f8 y <U8 en todas partes);

- Tiene algunos problemas de rendimiento en comparación con las matrices 'NumPy' normales;

- Se almacenan de forma contigua en la memoria, por lo que cada adición o eliminación de columna requiere la reasignación de toda la matriz;

- Todavía falta mucha funcionalidad de Pandas DataFrames.

Tu próximo intento probablemente sería almacenar cada columna como un vector 'NumPy' independiente. Y después de eso, tal vez envolverlas en un vector 'dict' para que sea más fácil restaurar la integridad de la "base de datos" si decides agregar o eliminar una fila o dos más adelante. Así es como se vería:

<img src="Img/Articulo_2_diagram_3.png">

Si lo has hecho, ¡felicitaciones! Has dado el primer paso para reimplementar Pandas. 

Ahora, aquí hay un par de ejemplos de lo que Pandas puede hacer por usted y que 'NumPy' no puede (o requiere un esfuerzo significativo para lograr).

### Exhibición de Pandas

Considere la siguiente tabla:

<img src="Img/Articulo_2_diagram_4.png">

Describe la variada línea de productos de una 'tienda online' con un total de cuatro productos distintos. A diferencia del ejemplo anterior, se puede representar con una matriz 'NumPy' o con un 'DataFrame de Pandas' igualmente bien. Pero veamos algunas operaciones comunes con él.

#### 1. Clasificación

La ordenación por columna es más legible con Pandas, como puedes ver a continuación:

<img src="Img/Articulo_2_diagram_5.png">

Aquí se calcula 'argsort(a[:,1])', la permutación que hace que la segunda columna de 'a' se ordene en orden ascendente y luego la externa 'a[…]' reordena las filas de 'a', según corresponda. Pandas puede hacerlo en un solo paso.

#### 2. Ordenar por varias columnas

Si necesitamos ordenar por columna de 'price' y desempatar usando la columna de 'weight', la situación empeora para 'NumPy':

<img src="Img/Articulo_2_diag_6.png">

Con 'NumPy', primero ordenamos por 'price' y luego aplicamos un segundo ordenamiento por 'weight'. Un algoritmo de ordenamiento estable garantiza que el resultado del primer ordenamiento no se pierda durante el segundo. Hay otras formas de hacerlo con 'NumPy', pero ninguna es tan simple y elegante como con Pandas.

#### 3. Agregar una columna

Agregar columnas es mucho mejor con Pandas, sintácticamente y arquitectónicamente:

<img src="Img/Articulo_2_diag_7.png">

'Pandas' no necesita reasignar memoria para toda la matriz como 'NumPy'; simplemente agrega una referencia a una nueva columna y actualiza un "registro" de los nombres de las columnas.

#### 4. Búsqueda rápida de elementos

Con las matrices 'NumPy', incluso si el elemento que busca es el primero, necesitará un tiempo proporcional al tamaño de la matriz para encontrarlo. Con 'Pandas', puede indexar las columnas que espera que se consulten con más frecuencia y reducir el tiempo de búsqueda a una constante.

<img src="Img/Articulo_2_diag_8.png">

La columna de índice tiene las siguientes limitaciones:

- Requiere memoria y tiempo para construirse.

- Es de sólo lectura (debe reconstruirse después de cada operación de adición o eliminación).

- No es necesario que los valores sean únicos, pero la aceleración solo ocurre cuando los elementos son únicos.

- Requiere calentamiento: la primera consulta es algo más lenta que en 'NumPy', pero las siguientes son significativamente más rápidas.

#### 5. Uniones por columnas

Si desea complementar una tabla con información de otra tabla basada en una columna común, 'NumPy' no es de mucha ayuda. 'Pandas' es mejor, especialmente para relaciones '1:n'.

<img src="Img/Articulo_2_diag_9.png">

En 'Pandas', 'join' tiene todos los modos de unión familiares: 'inner', 'left', 'right' y 'outer'.

#### 6. Agrupación por columnas

Otra operación habitual en el análisis de datos es la agrupación por columnas. Por ejemplo, para obtener la cantidad total vendida de cada producto, puede hacer lo siguiente:

<img src="Img/Articulo_2_diag_10.png">

Además de 'sum', Pandas admite todo tipo de funciones agregadas: 'mean', 'max', 'min', 'count', etc.

#### 7. Tablas dinámicas
Una de las funciones más potentes de Pandas es una tabla “pivotante”. Es algo así como proyectar un espacio multidimensional en un plano bidimensional.

<img src="Img/Articulo_2_diag_11.png">

In [49]:
import numpy as np
import pandas as pd

# Ejemplo de tablas Dinamicas
data={'client': ['John','John','Silvia','Silvia'],
      'product':['bananas','oranges','bananas','oranges'],
      'quantity':[5, 3, 4, 2]
}

df=pd.DataFrame(data)
print(df)

# Aplicando el método 'pivot' para crear una tabla dinámica
df.pivot(index='client', columns= 'product', values= 'quantity')


   client  product  quantity
0    John  bananas         5
1    John  oranges         3
2  Silvia  bananas         4
3  Silvia  oranges         2


product,bananas,oranges
client,,
John,5,3
Silvia,4,2


Si bien es posible implementarlo con 'NumPy', esta funcionalidad falta "de fábrica", aunque está presente en todas las principales bases de datos relacionales y aplicaciones de hojas de cálculo ( Excel , Google Sheets ).

Pandas también tiene una función 'df.pivot_table' que combina agrupación y pivoteo en una sola herramienta.

En este punto, es posible que te preguntes por qué alguien usaría 'NumPy' si Pandas es tan bueno. 'NumPy' no es mejor ni peor, solo tiene diferentes casos de uso:

- Números aleatorios (por ejemplo, para pruebas)

- Álgebra lineal (por ejemplo, para redes neuronales)

- Imágenes y pilas de imágenes (por ejemplo, para CNN)

- Personal científico de diferenciación, integración, trigonometría y otros.

En pocas palabras, las dos diferencias principales entre NumPy y Pandas son las siguientes:

<img src="Img/Articulo_2_diag_12.png">

Ahora veamos si esas características se consiguen a costa de una pérdida de rendimiento.

### La velocidad de los pandas

He evaluado 'NumPy' y 'Pandas' en una carga de trabajo típica de Pandas: 5 a 100 columnas; 10³ a 10⁸ filas; números enteros y flotantes. Estos son los resultados para 1 fila y 100 millones de filas:

<img src="Img/Articulo_2_diag_13.png">

¡Parece que en cada operación, 'Pandas' es más lento que 'NumPy'!

La situación (como era de esperar) no cambia cuando aumenta el número de columnas. En cuanto al número de filas, la dependencia (en escala logarítmica) se ve así:

<img src="Img/Articulo_2_diag_14.png">

Pandas parece ser 30 veces más lento que NumPy para matrices pequeñas (menos de cien filas) y tres veces más lento para matrices grandes (más de un millón de filas).

¿Cómo puede ser? Tal vez sea hora de enviar una solicitud de función para sugerir la reimplementación de Pandas a df.column.values.sum() través de 'df.column.sum()'? La propiedad values aquí proporciona acceso a la matriz 'NumPy' subyacente y da como resultado una aceleración de 3 a 30 veces.

La respuesta es no. Pandas es muy lento en esas operaciones básicas porque maneja correctamente los valores faltantes. Pandas necesita NaN (no-un-número) para toda esta maquinaria similar a una base de datos, como la agrupación y el pivoteo, además de que es algo común en el mundo real. En Pandas, se ha trabajado mucho para unificar el uso de 'NaN' en todos los tipos de datos admitidos. Por definición (impuesta a nivel de CPU), nan + cualquier cosa da como resultado nan. Entonces:

<img src="Img/Articulo_2_diag_15.png">

pero

<img src="Img/Articulo_2_diag_16.png">

Una comparación justa sería utilizar 'sum' 'np.nan' en lugar de np.sum, 'mean' 'np.nan' en lugar de 'np.mean' y así sucesivamente. Y de repente…

<img src="Img/Articulo_2_diag_17.png">

'Pandas' es 1,5 veces más rápido que 'NumPy' para matrices con más de un millón de elementos. Sigue siendo 15 veces más lento que 'NumPy' para matrices más pequeñas, pero por lo general no importa mucho si la operación se completa en '0,5 ms' o '0,05 ms': de todos modos es rápido.

En definitiva, si estás 100 % seguro de que no faltan valores en tus columnas, tiene sentido usar 'df.column.values.sum()' en lugar de 'df.column.sum()' para tener un aumento de rendimiento de 'x3' a 'x30'. En presencia de valores faltantes, la velocidad de Pandas es bastante decente e incluso supera a 'NumPy' para matrices enormes (más de 10⁶ elementos).

## Parte 2. Series e índices

<img src="Img/Articulo_2_diag_18.png">

'Series' es una contraparte de una matriz unidimensional en 'NumPy' y es un componente básico para un 'DataFrame' que representa su columna. Aunque su importancia práctica está disminuyendo en comparación con un 'DataFrame' (puede resolver perfectamente muchos problemas prácticos sin saber qué es una 'serie'), es posible que le resulte difícil comprender cómo funcionan los 'DataFrames' sin aprender primero sobre 'series' e 'índices'.

Internamente, 'Series' almacena los valores en un simple vector 'NumPy'. Como tal, hereda sus ventajas (diseño compacto de la memoria, acceso aleatorio rápido) y desventajas (homogeneidad de tipos, eliminaciones e inserciones lentas). Además, Series permite acceder a sus valores por etiqueta utilizando una estructura similar a un diccionario llamada 'index'. Las etiquetas pueden ser de cualquier tipo (comúnmente cadenas y marcas de tiempo). No necesitan ser únicas, pero se requiere la unicidad para aumentar la velocidad de búsqueda y se asume en muchas operaciones.

<img src="Img/Articulo_2_diag_19.png">

Como puedes ver, ahora cada elemento puede ser direccionado de dos maneras alternativas: por 'etiqueta' (usando el índice) y por 'posición' (sin usar el índice):

<img src="Img/Articulo_2_diag_20.png">

A veces, al direccionamiento "por posición" se le denomina "por índice posicional", lo que no hace más que aumentar la confusión.

Obviamente, un par de corchetes no es suficiente para esto. En particular:

- s[2:3] No es la forma más conveniente de abordar el elemento 'número 2'

- Si las etiquetas son números enteros, 's[1:3]' se vuelve ambiguo. Puede significar etiquetas '1' a '3' inclusive o índices posicionales '1' a '3' excluidos.

Para solucionar estos problemas, Pandas tiene dos "tipos" más de corchetes:

<img src="Img/Articulo_2_diag_21.png">

- .loc[] siempre utiliza 'etiquetas' e incluye ambos extremos del intervalo;

- .iloc[] siempre utiliza 'índices' posicionales y excluye el extremo derecho.

El propósito de usar corchetes en lugar de paréntesis aquí es obtener acceso a la conveniente segmentación de 'Python': puede usar dos puntos simples o dobles con el significado familiar de 'start:stop:step'. Como es habitual, falta de inicio ('stop') significa desde el inicio (hasta el 'stop') de la serie. El argumento de 'step' permite hacer referencia a filas pares con s.iloc[::2]y obtener elementos en orden inverso cons['Paris':'Oslo':-1]

También admiten la indexación booleana (indexación con una matriz de booleanos), como muestra esta imagen:

<img src="Img/Articulo_2_diag_22.png">

Y puedes ver cómo admiten la 'indexación elegante' (indexación con una matriz de números enteros) en esta imagen:

<img src="Img/Articulo_2_diag_23.png">

Lo peor de 'Series' es su representación visual: por alguna razón, no recibió una agradable apariencia de texto enriquecido, por lo que parece un ciudadano de 'segunda clase' en comparación con un 'DataFrame':

<img src="Img/Articulo_2_diag_24.png">

He 'parcheado' la serie para que se vea mejor, como se muestra a continuación:

<img src="Img/Articulo_2_diag_25.png">

In [9]:
import numpy as np
import pandas as pd
import pdi

pdi.patch_series_repr(footer=False)

pd.Series(['cat', 'dog', 'panda', 'cat', 'dragon'], index= list('abcde'), name='animal')

,animal
a,cat
b,dog
c,panda
d,cat
e,dragon


La 'línea vertical' significa que se trata de una serie, no de un 'marco de datos'. El pie de página está deshabilitado aquí, pero puede ser útil para mostrar tipos de datos, especialmente con elementos categóricos.

También puedes mostrar varias 'Series' o 'DataFrames' uno al lado del otro con 'pdi'.sidebyside(obj1, obj2, …):

<img src="Img/Articulo_2_diag_26.png">



pdi(significa 'p' y como se 'ilustra' ) es una biblioteca de código abierto en 'github'' con esta y otras funciones para este artículo. Para usarla, escribe

'pip install pandas-illustrated'

#### Índice

El objeto responsable de obtener los elementos de la serie (así como las filas y columnas del DataFrame) por etiqueta se denomina 'índice'. Es rápido: puede obtener el resultado en tiempo constante, ya sea que tenga cinco elementos o 5 mil millones de elementos.

'Indexes' una criatura verdaderamente polimórfica. De manera predeterminada, cuando crea una serie (o un DataFrame) sin argumentos 'index', se inicializa en un objeto perezoso similar al 'range()' de Python . Al igual que 'range()', apenas utiliza memoria y proporciona las etiquetas que coinciden con la indexación posicional. Creemos una serie de un millón de elementos:


In [20]:
s= pd.Series(np.zeros(10**6))
s.index

RangeIndex(start=0, stop=1000000, step=1)

In [34]:
import pandas as pd

indice = pd.RangeIndex(start=0, stop= 1000000, step= 1)
print(f"Memoria usada en 'bytes': {indice.memory_usage()}") # Número de bytes
                                                            # Lo mismo que para Series([0.]) 

Memoria usada en 'bytes': 132


Ahora, si eliminamos un elemento, el índice se transforma implícitamente en una estructura similar a un diccionario, de la siguiente manera:

In [41]:
import pandas as pd

s1= indice.drop(1) 
s1

Index([     0,      2,      3,      4,      5,      6,      7,      8,      9,
           10,
       ...
       999990, 999991, 999992, 999993, 999994, 999995, 999996, 999997, 999998,
       999999],
      dtype='int64', length=999999)

¡Esta estructura consume 8 MB de memoria! Para deshacerse de ella y volver a la estructura liviana de tipo rango, escriba:

In [47]:
import pandas as pd

s2= pd.Series(s1).reset_index(drop=True)
print(s2)
print()
s2 = pd.RangeIndex(start=0, stop= 999999, step= 1)
print(indice)
print()
print(f"Memoria usada en 'bytes': {s2.memory_usage()}")


0              0
1              2
2              3
3              4
4              5
           ...  
999994    999995
999995    999996
999996    999997
999997    999998
999998    999999
Length: 999999, dtype: int64

RangeIndex(start=0, stop=999999, step=1)

Memoria usada en 'bytes': 132


Si no conoce Pandas, es posible que se pregunte por qué Pandas no lo hizo por sí solo. Bueno, para las etiquetas no numéricas, es bastante obvio: ¿por qué (y cómo) Pandas, después de eliminar una fila, volvería a etiquetar todas las filas subsiguientes? Para las etiquetas numéricas, la respuesta es un poco más complicada.

Primero, como ya hemos visto, Pandas te permite referenciar filas puramente por posición, por lo que si quieres direccionar la fila número 5 después de eliminar la fila número 3, puedes hacerlo sin reindexar (para eso está iloc).

En segundo lugar, mantener las etiquetas originales es una forma de mantener una conexión con un momento del pasado, como un botón para "guardar el juego". Imagina que tienes una tabla grande con cien columnas y un millón de filas y necesitas encontrar algunos datos. Estás haciendo varias consultas una por una, cada vez acotando la búsqueda, pero mirando solo un subconjunto de las columnas, porque no es práctico ver todos los cien campos al mismo tiempo. Ahora que has encontrado las filas de interés, quieres ver toda la información sobre ellas en la tabla original. Un índice numérico te ayuda a obtenerla inmediatamente sin ningún esfuerzo adicional. Esquemáticamente, se ve así:

<img src="Img/Articulo_2_diag_27.png">

En general, es una buena idea mantener los valores de índice únicos. Por ejemplo, no obtendrá un aumento en la velocidad de búsqueda en presencia de valores duplicados en el índice. Pandas no tiene una "restricción única" como las bases de datos relacionales ( la característica aún es experimental), pero tiene funciones para verificar si los valores en el índice son únicos y para deshacerse de los duplicados de varias maneras.

A veces, una sola columna no es suficiente para identificar de forma única la fila. Por ejemplo, a veces sucede que las ciudades con el mismo nombre se encuentran en diferentes países o incluso en diferentes regiones del mismo país. Por lo tanto, (city, state) es un mejor candidato para identificar un lugar que una sola 'city'. En las bases de datos, se denomina "clave principal compuesta". En Pandas, se denomina 'MultiIndex' (consulte la Parte 4 a continuación) y cada columna dentro del índice se denomina "nivel".

Otra cualidad sustancial de un índice es que es inmutable. A diferencia de las columnas ordinarias en el 'DataFrame', no se puede modificar en el lugar. Cualquier cambio en el índice implica obtener datos del índice anterior, modificarlos y volver a adjuntar los nuevos datos como un nuevo índice. Por ejemplo, para convertir nombres de columnas en cadenas en el lugar (ahorra memoria), escribir 'df.columns = df.columns.astype(str)' o no en el lugar (útil para encadenar métodos). La mayoría de las veces, esto sucede de forma transparente (por ejemplo, al agregar o eliminar una columna), pero es la inmutabilidad la que no le permite simplemente escribir, por lo que debe recurrir a un método menos obvio '.df.set_axis(df.columns.astype(str), axis=1)' "df.City.name = 'city'" "df.rename(columns={'City': 'city'})"

El índice tiene un nombre (en el caso de MultiIndex, cada nivel tiene un nombre). Lamentablemente, este nombre no se usa lo suficiente en Pandas. Una vez que haya incluido la columna en el índice, 'df.column_name' ya no podrá utilizar la notación conveniente y deberá volver a la menos legible df.indexo a la más universal 'df.loc[]'. La situación empeora con 'MultiIndex'. Una excepción destacada es 'df.merge' que puede especificar la columna que se fusionará por nombre, sin importar si esta columna pertenece al índice o no.

Las columnas se etiquetan utilizando exactamente el mismo índice que las filas, aunque esto puede no ser evidente a partir de los argumentos del 'pd.DataFrameconstructor'.


#### Encontrar elemento por valor

Considere el siguiente objeto 'Serie':

<img src="Img/Articulo_2_diag_28.png">

'Index' ofrece una forma rápida y cómoda de buscar un valor por etiqueta. Pero ¿qué ocurre con la búsqueda de una etiqueta por valor?

<img src="Img/Articulo_2_diag_29.png">

He escrito un par de instrucciones llamados find() y findall() que son rápidos (ya que eligen automáticamente el comando real en función del tamaño de la serie) y más fáciles de usar. Así es como se ve el código:

In [ ]:
>>> import pdi
>>> pdi.find(s, 2)

'penguin'

>>> pdi.findall(s, 4)

Index(['cat', 'dog'], dtype= "object")

#### Valores faltantes

Los desarrolladores de Pandas han tenido especial cuidado con los valores faltantes. Normalmente, se recibe un dataframe con 'NaN' proporcionando una bandera a 'read_csv'. De lo contrario, se puede utilizar 'None' en el constructor o en un operador de asignación (funcionará a pesar de que se implementa de forma ligeramente diferente para diferentes tipos de datos), por ejemplo:

<img src="Img/Articulo_2_diag_31.png">

Lo primero que puedes hacer con los 'NaN' es saber si tienes alguno. Como se ve en la imagen de arriba, 'isna()' genera una matriz booleana y proporciona la cantidad total de valores faltantes..'sum()'

Ahora que sabes que están ahí, puedes optar por deshacerte de ellos eliminándolos, rellenándolos con un valor constante o interpolándolos, como se muestra a continuación:

<img src="Img/Articulo_2_diag_32.png">

Por otro lado, puedes seguir usándolos. La mayoría de las funciones de Pandas ignoran sin problemas los valores faltantes:

<img src="Img/Articulo_2_diag_33.png">

Las funciones más avanzadas ( median, rank, quantile, etc.) también lo hacen.

Las operaciones aritméticas se alinean con 'index':

<img src="Img/Articulo_2_diag_34.png">

Los resultados son inconsistentes en presencia de valores no únicos en el índice. No utilice operaciones aritméticas en series con un índice no único.

#### Comparaciones

Comparar matrices con valores faltantes es complicado.

Para empezar,
- Nonesiempre es igual a None,

- NaN( np.nantambién conocido como math.nanaka float('nan')) nunca es igual a NaN, y si todavía no es lo suficientemente extraño para ti...

- Al comparar pd.NAcon cualquier cosa siempre devuelve pd.NA:

In [ ]:
>>> None == None

True

>>> np.nan == np.nan

False

>>> np.NA == np.NA

<NA>

Cuando se trata de matrices, la cosa empeora:

In [ ]:
>>> np.all(pd.Series([1, None, 3.]) == pd.Series([1., None, 3.])) # Esto es 'np.nan'

False

>>> np.all(pd.Series(['a', None, 'c']) == pd.Series(['a', None, 'c'])) # Esto es 'None'

False

np.all(pd.Series([1, None, 3], dtype='Int64') == pd.Series([1, None, 3], dtype='Int64')) # Esto es pd.NA

True

Un método sencillo para compararlos correctamente es reemplazar todos estos tipos de pd.NA(que significa 'no disponible') con algo que seguramente no estará en la matriz. Por ejemplo, con '', -1 o ∞:

In [ ]:
>>> np.all(s1.fillna(np.inf) == s2.fillna(np.inf)) # Funciona para todos los tipos de datos

True

Una mejor manera es utilizar una función de comparación estándar de 'NumPy' o 'Pandas':

In [ ]:
>>> s = pd.Series([1., None, 3.]) 
>>> np.allclose(s.values, s.values, equal_nan=True) 

True 

>>> len(s.compare(s)) == 0 

True 

>>> pd.testing.assert_series_equal(s, s) # devuelve None

Aquí, 'compare' devuelve una lista de diferencias (un DataFrame, en realidad), 'allclose' devuelve 'array_equal' un valor booleano directamente y 'assert_series_equal' genera una excepción detallada cuando se encuentran diferencias.

Al comparar 'DataFrames' con tipos que no son "nativos" de 'NumPy' (por ejemplo 'object', 'Int64', , tipos mixtos, etc.), la comparación de 'NumPy' falla ( problema n.° 19205 ), mientras que 'Pandas' funciona perfectamente bien. Así es como se ve:

In [ ]:
>>> df = pd.DataFrame({'a': [1., Ninguno, 3.], 'b': ['x', Ninguno, 'z']}) 
>>> np.allclose(df.values, df.values, equal_nan=True) 

TypeError 

<...> 
>>> len(df.compare(df)) == 0 

True 

>>> pd.testing.assert_frame_equal(df, df) # devuelve Ninguno

Una ventaja agradable de usar funciones es su capacidad de comparar números flotantes correctamente (con tolerancias absolutas y relativas personalizables), tal como , de modo que dentro de las matrices 0,1+0,2 parece ser igual a 0,3: 'assert_anything_equal' 'np.allclose'

In [ ]:
>>> 0.1+0.2 == 0.3
 
Falso 

>>> assert_series_equal(pd.Series([0.1+0.2]), pd.Series([0.3])) # sin excepción

Tenga en cuenta que todos los métodos descritos anteriormente requieren que la forma (NumPy) o tanto la forma como el índice (Pandas) sean idénticos. De lo contrario,
los operadores de comparación 'compare' llegan hasta el punto de generar un 'ValueError' mientras 'np.array_equal' retorna 'False' y genera un con 'deltas' como de costumbre. La única excepción es que transmite las matrices antes de la comparación: 'assert_anything_equal' 'AssertionError' 'np.allclose'

In [ ]:
>>> np.array_equal(np.array([1, 1, 1]), np.array[1]) 

Falso 

>>> np.allclose(np.array([1, 1, 1]), np.array[1])
 
Verdadero 

>>> np.testing.assert_array_equal(np.array([1, 1, 1]), np.array[1]) 

Error de afirmación

#### Anexiones, inserciones, eliminaciones

Aunque se supone que los objetos Series son inmutables en tamaño, es posible agregar, insertar y eliminar elementos en el lugar, pero todas esas operaciones son:

- Lento, ya que requieren reasignar memoria para todo el objeto y actualizar el índice;

- dolorosamente inconveniente

Aquí hay una forma de insertar un valor y dos formas de eliminar los valores:

<img src="Img/Articulo_2_diag_35.png">

El segundo método para eliminar valores (a través de drop) es más lento y puede generar errores complejos en presencia de valores no únicos en el índice.

Pandas tiene el método, pero solo puede insertar columnas (no filas) en un marco de datos (y no funciona en absoluto con series).df.insert

Otro método para agregar e insertar es cortar el DataFrame con iloc, aplicar las conversiones necesarias y luego volver a colocarlo con concat. He implementado una función llamada insertque automatiza el proceso:

<img src="Img/Articulo_2_diagram_36.png">

Tenga en cuenta que (al igual que en df.insert) el lugar a insertar, viene dado por una posición 0<=i<=len(s), no por la etiqueta del elemento del índice.

Puede proporcionar una etiqueta para un nuevo elemento. Para un índice no numérico, es obligatorio. Por ejemplo:

<img src="Img/Articulo_2_diag_37.png">

Para especificar el punto de inserción por etiqueta, puede combinar 'pdi.find' con 'pdi.insert', como se muestra a continuación:

<img src="Img/Articulo_2_diag_38.png">

Tenga en cuenta que, a diferencia de 'df.insert', 'pdi.insert' devuelve una copia en lugar de modificar la 'Serie/DataFrame' en el lugar.

#### Estadistica

Pandas ofrece un espectro completo de funciones estadísticas que pueden brindarle una idea de lo que hay en una serie o un marco de datos de un millón de elementos sin tener que desplazarse manualmente por los datos.

Todas las funciones estadísticas de Pandas ignoran los 'NaN', como puedes ver a continuación:

<img src="Img/Articulo_2_diag_39.png">

Tenga en cuenta que 'Pandas std' ofrece resultados diferentes a los de 'NumPy.std'.

Dado que se puede acceder a cada elemento de una serie mediante una etiqueta o un índice posicional, existe una función hermana para argmin (argmax) llamada idxmin ( idxmax), que se muestra en la imagen:

<img src="Img/Articulo_2_diag_40.png">

A continuación se muestra una lista de funciones estadísticas autodescriptivas de Pandas como referencia:

- std, desviación estándar de la muestra;

- var, varianza imparcial;

- sem, error estándar no sesgado de la media;

- quantile, cuantil de muestra ( s.quantile(0.5) ≈ s.median());

- mode, el(los) valor(es) que aparecen con más frecuencia;

- nlargest y nsmallest, por defecto, en orden de aparición;

- diff, primera diferencia discreta;

- cumsum y cumprod, suma acumulada, y producto;

- cummin y cummax, mínimo y máximo acumulativo.

Y algunas funciones estadísticas más especializadas:

- pct_change, cambio porcentual entre el elemento actual y el anterior;

- skew, asimetría imparcial (tercer momento);

- kurt o bien kurtosis, curtosis imparcial (cuarto momento);

- cov, corr y autocorr, covarianza, correlación y autocorrelación;

- rolling, weighted, y ventanas exponentially weighted.

#### Datos duplicados

Se tiene especial cuidado en detectar y tratar los datos duplicados, como se puede ver en la imagen:

<img src="Img/Articulo_2_diag_41.png">

'drop_duplicates' y 'duplicated' puede mantener la última ocurrencia en lugar de la primera.

De hecho, estas dos funciones son complementarias entre sí:
df.drop_duplicates() == df[~df.duplicated()]

Tenga en cuenta que 's.unique()' es más rápido que 'np.unique' (O(N) vs O(NlogN)) y conserva el orden en lugar de devolver los resultados ordenados como np.unique lo hace.

Los valores faltantes se tratan como valores ordinarios, lo que a veces puede generar resultados sorprendentes.

<img src="img/Articulo_2_diag_42.png">



Si desea excluir los 'NaN', debe hacerlo explícitamente. En este ejemplo en particular, 's.dropna().is_unique == True'.

También existe una familia de propiedades monótonas con nombres autodescriptivos:

- s.is_monotonic_increasing,

- s.is_monotonic_decreasing, y, de manera bastante inesperada,

- s.is_monotonic — que es un sinónimo 's.is_monotonic_increasing()' y devuelve 'False' una 
serie decreciente monótona (afortunadamente, obsoleto y eliminado en Pandas 2.0)

No hay funciones documentadas que verifiquen la monotonía estricta, pero puedes construir una fácilmente combinándolas con 's.unique', por ejemplo, para verificar si saumenta estrictamente de manera monótona, escribe
's.unique' y 's.is_monotonic_increasing'

#### Cadenas y expresiones regulares

Prácticamente todos los métodos de cadena de Python tienen una versión vectorizada en Pandas:

<img src="Img/Articulo_2_diag_43.png">

Cuando una operación de este tipo devuelve varios valores, tienes varias opciones sobre cómo utilizarlos:

<img src="Img/Articulo_2_diag_44.png">

Si conoce expresiones regulares, Pandas también tiene versiones vectorizadas de las operaciones comunes con ellas:

<img src="Img/Articulo_2_diag_45.png">

#### Group by

Una operación común en el procesamiento de datos es calcular algunas estadísticas no sobre todo el conjunto de datos sino sobre ciertos grupos de ellos. El primer paso es construir un objeto perezoso proporcionando criterios para dividir una serie (o un marco de datos) en grupos. Este objeto perezoso no tiene una representación significativa, pero puede ser:

- iterado (produce la clave de agrupación y la subserie correspondiente, ideal para depuración):

<img src="Img/Articulo_2_diag_46.png">

- Se consulta de la misma manera que las series ordinarias para obtener una determinada propiedad de cada grupo (es más rápido que la iteración):

<img src="Img/Articulo_2_diag_47.png">

En este ejemplo, dividimos la serie en tres grupos según la parte entera de dividir los valores por 10. Para cada grupo, solicitamos la suma de los elementos, la cantidad de elementos y el valor promedio en cada grupo.

Además de esas funciones de agregación, puedes acceder a elementos específicos en función de su posición o valor relativo dentro de un grupo. Así es como se ve:

<img src="Img/Articulo_2_diag_48.png">

También puede calcular varias funciones en una sola llamada con g.agg(['min', 'max'])o mostrar un montón de funciones estadísticas a la vez con 'g.describe()'.

Si esto no es suficiente, también puedes pasar los datos a través de tu propia función de Python. Puede ser:

- Una función 'f' que acepta un grupo x(un objeto Serie) y genera un único valor (por ejemplo, sum()) cong.apply(f)

- Una función 'f' que acepta un grupo x(un objeto Serie) y genera un objeto Serie del mismo tamaño que x(por ejemplo, cumsum()) cong.transform(f)

<img src="Img/Articulo_2_diag_49.png">

En los ejemplos anteriores, los datos de entrada están ordenados. Esto no es necesario para 'groupby'. En realidad, funciona igual de bien si los elementos del grupo no se almacenan de forma consecutiva, por lo que es más cercano a 'collections.defaultdict' que a 'itertools.groupby'. Y siempre devuelve un índice sin duplicados.

<Img src="Img/Articulo_2_diag_50.png">

A diferencia de 'defaultdict' la cláusula 'GROUP BY' de una base de datos relacional, Pandas 'groupby' ordena los resultados por nombre de grupo. Puede desactivarse con 'sort='False', como verá en el código:

In [ ]:
>>> s = pd.Series([1, 3, 20, 2, 10])
>>> for k, v in s.groupby(s//10, sort=False):
       print(k, v.tolist())
0 [1, 3, 2]
2 [20]
1 [10]

Descargo de responsabilidad: En realidad, 'g.apply(f)' es más versátil de lo descrito anteriormente:

- Si f(x) devuelve una serie del mismo tamaño que x, puede imitar la transformación

- Si f(x) devuelve una serie de diferente tamaño o un marco de datos, da como resultado una serie con un Multiindex correspondiente.

Pero los documentos advierten que esos usos pueden ser más lentos que los métodos correspondientes transform, aggasí que tenga cuidado.

## Parte 3. DataFrame

<img src="Img/Articulo_2_diag_51.png">

La estructura de datos principal de Pandas es un 'DataFrame'. Agrupa una matriz bidimensional con etiquetas para sus filas y columnas. Consta de una serie de objetos Series (con un índice compartido), cada uno de los cuales representa una sola columna y posiblemente tenga diferentes tipos de datos.

#### Lectura y escritura de archivos CSV

Una forma común de construir un 'DataFrame' es leyendo un archivo CSV (valores separados por comas), como muestra esta imagen:

<img src="Img/Articulo_2_diag_52.png">

La función es una herramienta totalmente automatizada y muy personalizable. Si quieres aprender una sola cosa sobre Pandas, aprende a usarlo. ¡ Valdrá la pena!:) 'pd.read_csv()read_csv'

A continuación se muestra un ejemplo de análisis de un archivo CSV no estándar:

<img src="Img/Articulo_2_diag_53.png">

Y una breve descripción de algunos de los argumentos:

<img src="Img/Articulo_2_diag_54.png">

Dado que CSV no tiene una especificación estricta, a veces es necesario un poco de ensayo y error para leerlo correctamente. Lo bueno 'read_csv' es que detecta automáticamente muchas cosas, entre ellas:

- Nombres y tipos de columnas,

- Representación de booleanos,

- Representación de valores faltantes, etc.

Al igual que con cualquier automatización, es mejor asegurarse de que haya hecho lo correcto. Si los resultados de escribir simplemente 'df' en una celda de Jupyter resultan ser demasiado largos (o demasiado incompletos), puede probar lo siguiente:

- df.head(5) o df[:5] muestra las primeras cinco filas

- df.dtypes devuelve los tipos de columna

- df.shape devuelve el número de filas y columnas

- df.info() Resume toda la información relevante

Es una buena idea establecer una o varias columnas como índice. La siguiente imagen muestra este proceso:

<img src="Img/Articulo_2_diag_55.png">



Index tiene muchos usos en Pandas:

- Hace que las búsquedas por columnas indexadas sean más rápidas;

- Las operaciones aritméticas, el apilamiento y la unión se alinean por índice, etc.

Todo esto se produce a costa de un consumo de memoria algo mayor y una sintaxis un poco menos obvia.

#### Construyendo un DataFrame

Otra opción es construir un marco de datos a partir de datos ya almacenados en la memoria. Su constructor es tan extraordinariamente omnívoro que puede convertir (o encapsular) cualquier tipo de datos que le introduzcas:

<img src="Img/Articulo_2_diag_56.png">

En el primer caso, en ausencia de etiquetas de fila, Pandas etiquetó las filas con números enteros consecutivos. En el segundo caso, hizo lo mismo con las filas y las columnas. Siempre es una buena idea proporcionar a Pandas los nombres de las columnas en lugar de las etiquetas de números enteros (usando el columnsargumento) y, a veces, los nombres de las filas (usando el indexargumento, aunque rowspuede sonar más intuitivo). Esta imagen te ayudará:

<img src="Img/Articulo_2_diag_57.png">

Para asignar un nombre a la columna de índice, escriba df.index.name = 'city_name' o usepd.DataFrame(..., index=pd.Index(['Oslo', 'Vienna', 'Tokyo'], name='city_name')).

La siguiente opción es construir un DataFrame a partir de un diccionario de vectores 'NumPy' o una matriz NumPy 2D:

<img src="Img/Articulo_2_diag_58.png">

Observe cómo los population valores se convirtieron en flotantes en el segundo caso. En realidad, esto ocurrió antes, durante la construcción de la matriz 'NumPy'. Otra cosa que debe tener en cuenta aquí es que la construcción de un marco de datos a partir de una matriz 'NumPy' 2D es una vista predeterminada. Eso significa que al cambiar los valores en la matriz original se cambia el marco de datos y viceversa. Además, ahorra memoria.

Este modo también se puede habilitar en el primer caso (un diccionario de vectores NumPy) configurando copy=False. Sin embargo, es muy frágil. Operaciones simples pueden convertirlo en una copia sin previo aviso.

Dos opciones más (menos útiles) para crear un DataFrame son:

- De una lista de diccionarios (donde cada diccionario representa una sola fila, sus claves son nombres de columnas y sus valores son los valores de celda correspondientes)

- De un 'dict' de Series (donde cada Serie representa una columna; devuelve una copia por defecto, se le puede indicar que devuelva una vista con copy=False).

Si registra datos de transmisión "sobre la marcha", su mejor opción es utilizar un diccionario de listas o una lista de listas porque Python preasigna espacio de manera transparente al final de una lista para que las adiciones sean rápidas. Ni las matrices de 'NumPy' ni los DataFrame de Pandas lo hacen. Otra posibilidad (si conoce la cantidad de filas de antemano) es preasigna memoria manualmente con algo como DataFrame(np.zeros).

#### Operaciones básicas con DataFrames

Lo mejor de DataFrame (en mi opinión) es que puedes:

- Acceder fácilmente a sus columnas, por ejemplo, df.areadevuelve valores de columna (o alternativamente, df[‘area’] — bueno para nombres de columnas que contienen espacios)

- Operar las columnas como si fueran variables independientes, por ejemplo, después de df.population /= 10**6 almacenar la población en millones, y el siguiente comando crea una nueva columna llamada 'densidad' calculada a partir de los valores de las columnas existentes:

<img src="Img/Articulo_2_diag_59.png">

Tenga en cuenta que al crear una nueva columna, los corchetes son obligatorios incluso si su nombre no contiene espacios.

Además, puedes utilizar operaciones aritméticas en columnas incluso de diferentes DataFrames, siempre que sus filas tengan etiquetas significativas, como se muestra a continuación:

<img src="Img/Articulo_2_diag_60.png">

In [53]:
import numpy as np
import pandas as pd

# Construyendo un DataFrame

# De un dicionario de lista
df= pd.DataFrame({'name':['Oslo','Vienna','Tokyo'],
              'population':[698660,1911191,14043239],
              'area':[480.8, 414.8, 2194.1]
    
})

# De una lista de lista
df1= pd.DataFrame([['Oslo', 698660, 480.8],
                   ['Vienna', 1911191, 414.8],
                   ['Tokyo', 14043239, 2194.1]  
])

print("'DataFrame' a partir de un 'Diccionario' de 'Lista'")
print(df)
print()
print("'DataFrame' a partir de una 'Lista' de 'Lista'")
print(df1)

'DataFrame' a partir de un 'Diccionario' de 'Lista'
     name  population    area
0    Oslo      698660   480.8
1  Vienna     1911191   414.8
2   Tokyo    14043239  2194.1

'DataFrame' a partir de una 'Lista' de 'Lista'
        0         1       2
0    Oslo    698660   480.8
1  Vienna   1911191   414.8
2   Tokyo  14043239  2194.1


#### Indexación de DataFrames

Como ya hemos visto en la sección Series, los corchetes comunes simplemente no son suficientes para satisfacer todas las necesidades de indexación. No se puede acceder a las filas por etiquetas, no se puede acceder a las filas disjuntas por índice posicional y ni siquiera se puede hacer referencia a una sola celda, ya que df['x', 'y'] está reservada para MultiIndex.

<img src="Img/Articulo_2_diag_61.png">

Para satisfacer esas necesidades, los DataFrames, al igual que las series, tienen dos modos de indexación alternativos: 'loc' para indexar por etiquetas y 'iloc' para indexar por índice posicional.

<img src="Img/Articulo_2_diag_62.png">

En Pandas, hacer referencia a varias filas o columnas es una copia, no una vista. Pero es un tipo especial de copia que permite realizar asignaciones en su totalidad:

- df.loc['a']=10 Trabajos (una sola fila se puede escribir como un todo)

- df.loc['a']['A']=10 Trabajos (el acceso al elemento se propaga al original df)

- df.loc['a':'b'] = 10 Trabajos (asignar a un subarreglo como un trabajo completa)

- df.loc['a':'b']['A'] = 10 no (asignar a sus elementos no lo hace).

En el último caso, el valor solo se establecerá en una copia de una porción y no se reflejará en el original df(se mostrará una advertencia en consecuencia).

Dependiendo de la situación existen diferentes soluciones:

1. Quiere cambiar el marco de datos original df. Entonces use
df.loc['a':'b', 'A'] = 10

2. Ha realizado la copia intencionalmente y desea trabajar en esa copia:
df1 = df.loc['a':'b']; df1['A']=10 # SettingWithCopy warning para deshacerse de una advertencia en esta situación, conviértala en una copia real:

df1 = df.loc['a':'b'].copy(); df1['A']=10

Pandas también admite una sintaxis 'NumPy' conveniente para la indexación booleana.

<img src="Img/Articulo_2_diag_63.png">

Al utilizar varias condiciones, estas deben estar entre paréntesis, como puedes ver a continuación:

<img src="Img/Articulo_2_diag_64.png">

Cuando esperas que se devuelva un único valor, necesitas tener especial cuidado.

<img src="Img/Articulo_2_diag_65.png">


Dado que potencialmente podría haber varias filas que coincidan con la condición, 'loc' se devolvió una serie. Para obtener un valor escalar, puede utilizar:

- float(s) o una más universal s.item()que generará ValueError a menos que haya exactamente un valor en la serie

- s.iloc[0] que solo lanzará una excepción cuando no se encuentre nada; además, es el único que admite asignaciones: df[…].iloc[0] = 100, pero seguramente no lo necesitas cuando quieres modificar todas las coincidencias: df[…] = 100.

Alternativamente, puede utilizar consultas basadas en cadenas:

- df.query('name=="Vienna"')

- df.query('population>1e6 and area<1000')

Son más cortos, funcionan muy bien con 'MultiIndex' y los operadores lógicos tienen prioridad sobre los operadores de comparación (se requieren menos paréntesis), pero solo pueden filtrar por filas y no se puede modificar el 'DataFrame' a través de ellos.

Varias bibliotecas de terceros le permiten usar la sintaxis SQL para consultar los DataFrames directamente (duckdb) o indirectamente copiando el dataframe a 'SQLite' y envolviendo los resultados nuevamente en objetos Pandas (pandasql). Como era de esperar, el método directo es más rápido.

In [71]:
# Ejemplo de Indexacion de diccionario

print(df)
print()
print('Nombres de paises con poblaciones mayores de un millon de habitantes')
print(df.loc[df.population>10**6, ['name']])
print()
print('Paises con poblaciones mayores de un millon de habitantes')
print('y área menor de 1000')
print(df.loc[(df.population>10**6) & (df.area<1000), ['name', 'population', 'area']])
print()
print("Area de 'Viena'")
print(df.loc[df.name=='Vienna', ['area']])
print()


     name  population    area
0    Oslo      698660   480.8
1  Vienna     1911191   414.8
2   Tokyo    14043239  2194.1

Nombres de paises con poblaciones mayores de un millon de habitantes
     name
1  Vienna
2   Tokyo

Paises con poblaciones mayores de un millon de habitantes
y área menor de 1000
     name  population   area
1  Vienna     1911191  414.8

Area de 'Viena'
    area
1  414.8



#### Aritmética de DataFrame

Puede aplicar operaciones ordinarias como sumar, restar, multiplicar, dividir, módulo, potencia, etc., a marcos de datos, series y combinaciones de los mismos.

Todas las operaciones aritméticas se alinean con las etiquetas de filas y columnas:

<img src="Img/Articulo_2_diag_66.png">

En operaciones mixtas entre 'DataFrames' y 'Series', la Serie (Dios sabe por qué) se comporta (y transmite) como un vector de fila y se alinea en consecuencia:

<img src="Img/Articulo_2_diag_67.png">

Probablemente para mantenernos en línea con las listas y los vectores 'NumPy' 1D (que no están alineados por etiquetas y se espera que tengan el mismo tamaño que si el 'DataFrame' fuera una simple matriz 'NumPy' 2D):

<img src="Img/Articulo_2_diag_68.png">

Entonces, en el desafortunado (y, por coincidencia, ¡el más habitual!) caso de dividir un marco de datos por una serie de vectores de columna, debes usar métodos en lugar de operadores, como puedes ver a continuación:

<img src="Img/Articulo_2_diag_69.png">

Debido a esta decisión cuestionable, siempre que necesites realizar una operación mixta entre un DatFrame y una serie tipo columna, debes buscarla en la documentación (o memorizarla):

<img src="Img/Articulo_2_diag_70.png">

#### Combinación de DataFrames

Pandas tiene una gran cantidad de funciones:

— concat(concatenación),

— join, merge y merge_asof(fusiones de estilo base de datos),

— combine_first, combiney update(superposición de un 'df' sobre el otro)
que básicamente hacen lo mismo: combinan información de varios dataframes en uno. Pero cada una de ellas lo hace de manera ligeramente diferente, ya que están diseñadas para diferentes casos de uso:

<img src="Img/Articulo_2_diag_71.png">

Ahora veremos algunos detalles intrincados sobre las diferentes opciones, modos y cómo funcionan con 'MultiIndex'.


#### Apilamiento vertical

Esta es probablemente la forma más sencilla de combinar dos o más marcos de datos en uno: se toman las filas del primero y se añaden las filas del segundo al final. Para que funcione, esos dos 'DataFrame' deben tener (aproximadamente) las mismas columnas. Esto es similar a lo que ocurre vstacken 'NumPy', como se puede ver en la imagen:

<img src="Img/Articulo_2_diag_72.png">

Tener valores duplicados en el índice es malo. Puede encontrarse con varios tipos de problemas (vea el ejemplo de "eliminación" a continuación). Incluso si no le importa el índice, intente evitar que tenga valores duplicados en él:

- O bien usa 'reset_index=True' argumento

- Llamada 'df.reset_index(drop=True)' para reindexar las filas de '0' a 'len(df)-1',

- Utilice el keysargumento para resolver la ambigüedad con MultiIndex (ver más abajo).

Si las columnas de los DataFrames no coinciden perfectamente entre sí (aquí no cuenta un orden diferente), Pandas puede tomar la intersección de las columnas ( kind='inner’, el valor predeterminado) o insertar NaN para marcar los valores faltantes ( kind='outer'):

<img src="Img/Articulo_2_diag_73.png">


#### Apilamiento horizontal

'concat' También puede realizar apilamiento 'horizontal' (similar 'hstack' a 'NumPy'):

<img src="Img/Articulo_2_diag_74.png">

'join' es más configurable que 'concat': en particular, tiene cinco modos de unión en lugar de solo dos de concat. Consulte la sección "Unión de relaciones 1:1" a continuación para obtener más detalles.

#### Apilamiento mediante 'MultiIndex'

Si las etiquetas de fila y columna coinciden, concatpermite realizar un equivalente 'MultiIndex' de apilamiento vertical (como 'dstack' en 'NumPy'):

<img src="Img/Articulo_2_diag_75.png">

Si la fila y/o las columnas se superponen parcialmente, 'Pandas' alineará los nombres en consecuencia, y probablemente eso no sea lo que desea:

<img src="Img/Articulo_2_diag_76.png">

En general, si las etiquetas se superponen, significa que los marcos de datos están relacionados de alguna manera entre sí, y las relaciones entre entidades se describen mejor utilizando la terminología de bases de datos relacionales.

#### La relación 1:1

<img src="Img/Articulo_2_diag_77.png">

Esto sucede cuando la información sobre el mismo grupo de objetos se almacena en varios 'DataFrames' diferentes y desea combinarlos en un solo 'DataFrame'.

Si la columna que desea fusionar no está en el índice, utilice 'merge'.

<img src="Img/Articulo_2_diag_78.png">

Lo primero que hace es descartar todo lo que se encuentre en el índice. Luego realiza la unión. Por último, renumera los resultados de 0 a n-1.

Si la columna ya está en el índice, puedes usar join(que es solo un alias para merge con 'left_index' o 'right_index' establecido en 'True' y diferentes valores predeterminados).

<img src="Img/Articulo_2_diag_79.png">

Como puede ver en este caso simplificado (consulte "unión externa completa" más arriba), Pandas es bastante flexible en cuanto a mantener el orden de las filas en comparación con las bases de datos relacionales. Las uniones externas izquierdas y derechas tienden a ser más predecibles que las uniones internas y externas (al menos, hasta que hay valores duplicados en la columna que se va a fusionar). Por lo tanto, si desea un orden de filas garantizado, tendrá que ordenar los resultados de forma explícita o usar 'Categorical Index' ( pdi.lock puede ayudarlo con eso).

#### La relación 1:n

<img src="Img/Articulo_2_diag_89.png">

Esta es la relación más utilizada en el diseño de bases de datos, donde una fila de la 'tabla A' (por ejemplo, 'Estado') se puede vincular a varias filas de la 'tabla B' (por ejemplo, Ciudad), pero cada fila de la 'tabla B' solo se puede vincular a una fila de la 'tabla A' (una ciudad solo puede estar en un solo estado, pero un estado consta de varias ciudades).

Al igual que en las 'relaciones 1:1', para unir un par de tablas relacionadas '1:n' en 'Pandas', tiene dos opciones. Si la columna que se va a fusionar no está en el índice y no tiene problemas en descartar todo lo que esté en el índice de ambas tablas, utilice merge, por ejemplo:

<img src="Img/Articulo_2_diag_81.png">

Como ya hemos visto, mergemantiene el orden de las filas de forma menos rigurosa que, por ejemplo, Postgres. La declaración "preservar el orden de las claves" de la documentación solo se aplica a left_index=Trueand/or right_index=True(para eso joines un alias) y solo en ausencia de valores duplicados en la columna que se va a fusionar. Por eso mergeand jointiene un sortargumento.

Ahora, si la columna a fusionar ya está en el índice del DataFrame correcto, use join(o mergecon right_index=True, que es exactamente lo mismo):

<img src="Img/Articulo_2_diag_82.png">

Esta vez, 'Pandas' mantuvo intactos los valores de índice del marco de datos izquierdo y el orden de las filas.

Nota: tenga cuidado, si la segunda tabla tiene valores de índice duplicados, terminará con valores de índice duplicados en el resultado, ¡incluso si el índice de la tabla izquierda es único!

A veces, los 'DataFrames' unidos tienen columnas con el mismo nombre. Tanto 'merge' y 'join' tienen una forma de resolver la ambigüedad, pero la sintaxis es ligeramente diferente (además, de forma predeterminada, 'merge' la resolverá con '_x', '_y’ mientras que 'join' generará una excepción), como puede ver en la imagen a continuación:

<img src="Img/Articulo_2_diag_83.png">

Para resumir:

- 'merge' se une a columnas que no son de índice, 'join' requiere que la columna 'derecha' esté indexada;

- 'merge'descarta el índice del 'DataFrame izquierdo', 'join' lo conserva;

- por defecto, 'merge' realiza un 'inner join', 'join' hace una unión 'left outer';

- 'merge' no mantiene el orden de las filas, 'join' las conserva (se aplican algunas restricciones);

- 'join' es un alias para 'merge' con 'left_index=True' y/o 'right_index=True'.

In [87]:
import numpy as np
import pandas as pd

# RELACIONES
df= pd.DataFrame({'name': ['Oslo', 'Vienna'],
              'population':[698660,1911191]
    
})

df1= pd.DataFrame({'name': ['Vienna', 'Tokyo'],
                   'area': [414.8, 2194.1]
    
})

cities= pd.DataFrame({'city': ['San Francisco', 'Miami', 'Washington', 'Los Angeles'],
                   'state-code': ['CA', 'FL', 'DC', 'CA']
    
})

states= pd.DataFrame({'code': ['CA', 'FL', 'TX'],
                     'state': ['California', 'Florida', 'Texas']
    
})


#1:1
print(df)
print()
print(df1)
print()
print("RELACION 1: (merge)")
print("Relación 1:1 'inner join'")
print(df.merge(df1, on='name'))
print()
print("Relación 1:1 'left'")
print(df.merge(df1, how='left'))
print()
print("Relación 1:1 'outer'")
print(df.merge(df1, how='outer'))
print()
print("Relación 1:1 'right'")
df.merge(df1, how='right')

     name  population
0    Oslo      698660
1  Vienna     1911191

     name    area
0  Vienna   414.8
1   Tokyo  2194.1

RELACION 1: (merge)
Relación 1:1 'inner join'
     name  population   area
0  Vienna     1911191  414.8

Relación 1:1 'left'
     name  population   area
0    Oslo      698660    NaN
1  Vienna     1911191  414.8

Relación 1:1 'outer'
     name  population    area
0    Oslo    698660.0     NaN
1   Tokyo         NaN  2194.1
2  Vienna   1911191.0   414.8

Relación 1:1 'right'


,name,population,area
0,Vienna,1911191.0,414.8
1,Tokyo,NaN,2194.1


In [89]:
print("RELACION 1:n")
print(cities)
print()
print(states)
print()
print("Relación con 'merge' cuando los códigos de los DataFrame difieren:")
df2= cities.merge(states, left_on= 'state-code', right_on='code')
print(df2)
print()
print("Filtrando la 'Ciudad' y el 'Estado'")
print(df2.filter(['city','state']))

RELACION 1:n
            city state-code
0  San Francisco         CA
1          Miami         FL
2     Washington         DC
3    Los Angeles         CA

  code       state
0   CA  California
1   FL     Florida
2   TX       Texas

Relación con 'merge' cuando los códigos de los DataFrame difieren:
            city state-code code       state
0  San Francisco         CA   CA  California
1          Miami         FL   FL     Florida
2    Los Angeles         CA   CA  California

Filtrando la 'Ciudad' y el 'Estado'
            city       state
0  San Francisco  California
1          Miami     Florida
2    Los Angeles  California


In [91]:
print("HACIENDO USO DEL JOIN")
df3= cities.join(states)
# Relaciona por los índices
print(df3)


HACIENDO USO DEL JOIN
            city state-code code       state
0  San Francisco         CA   CA  California
1          Miami         FL   FL     Florida
2     Washington         DC   TX       Texas
3    Los Angeles         CA  NaN         NaN


#### Uniones múltiples

Como se explicó anteriormente, cuando 'join' se ejecuta contra dos 'DataFrames', por ejemplo 'df.join(df1)', actúa como un alias de 'merge'. Pero 'join' también tiene un modo de "unión múltiple", que, a su vez, es un alias de concat(axis=1).

<img src="Img/Articulo_2_diag_84.png">

Este modo es algo limitado en comparación con el modo normal:

- no proporciona un medio para la resolución de columnas duplicadas;

- Sólo funciona para 'relaciones 1:1' (uniones de índice a índice).

Por lo tanto, se supone que varias 'relaciones 1:n' se deben unir una por una. El repositorio 'pandas-illustrated' también tiene una herramienta para eso, como puede ver a continuación:

<img src="Img/Articulo_2_diag_85.png">

'pdi.join' es un contenedor simple 'join' que acepta listas en argumentos 'on', 'how' y 'suffixes'  para que puedas hacer varias uniones en un comando. Al igual que con la unión original, 'on' las columnas pertenecen al 0 primer DataFrame0  y los demás 'DataFrames' se unen en función de sus índices.

#### Inserta y elimina

Dado que un 'DataFrame' es una colección de columnas, es más fácil aplicar estas operaciones a las filas que a las columnas. Por ejemplo, la inserción de una columna siempre se realiza en el lugar, mientras que la inserción de una fila siempre da como resultado un nuevo 'DataFrame', como se muestra a continuación:

<img src="Img/Articulo_2_diag_86.png">

Eliminar columnas normalmente no genera preocupaciones, excepto que del df['D']funciona mientras del 'df.D' no funciona (limitación en el nivel de Python).

<img src="Img/Articulo_2_diag_87.png">

Eliminar filas con 'drop' es sorprendentemente lento y puede generar errores complejos si las etiquetas sin procesar no son únicas. Por ejemplo:

<img src="Img/Articulo_2_diag_88.png">

Una solución sería utilizar 'ignore_index=True', que indica 'concat' que se deben restablecer los nombres de las filas después de la concatenación:

<img src="Img/Articulo_2_diag_90.png">

En este caso, namesería útil configurar la columna como índice, pero no así para filtros más complejos.

Otra solución rápida, universal e incluso que funciona con nombres de filas duplicados es utilizar la indexación en lugar de la eliminación. Puede negar la condición manualmente o utilizar una automatización (de una sola línea) de la biblioteca "pdi":

<img src="Img/Articulo_2_diag_91.png">

#### groupby

Esta operación ya se ha descrito en detalle en la sección 'Series', pero 'DataFrame' 'groupby' tiene un par de trucos específicos además de eso.

Primero, puedes especificar la columna a agrupar usando solo un nombre, como se muestra en la siguiente imagen:

<img src="Img/Articulo_2_diag_92.png">

Sin 'as_index=False', Pandas convierte la columna por la que se realizó la agrupación en la columna de índice. Si esto no es deseable, puede utilizar 'reset_index()' o especificar 'as_index=False'.

Por lo general, hay más columnas en el DataFrame de las que desea ver en el resultado. De manera predeterminada, Pandas suma todo lo que sea remotamente sumable, por lo que deberá limitar su elección, como se muestra a continuación:

<img src="Img/Articulo_2_diag_93.png">

Tenga en cuenta que, al sumar sobre una sola columna, obtendrá una serie en lugar de un DataFrame. Si, por algún motivo, desea un DataFrame, puede:

- Utilice corchetes dobles: 'df.groupby('product')[['quantity']].sum()' o

- convertir explícitamente: 'df.groupby('product')['quantity'].sum().to_frame()'

Cambiar al índice numérico también lo convertirá en un DataFrame:

- 'df.groupby('product', as_index=False)['quantity'].sum()' o

- 'df.groupby('product')['quantity'].sum().reset_index()'

Pero a pesar de la apariencia inusual, en muchos casos una Serie se comporta igual que un DataFrame, por lo que tal vez un "lavado de cara" 'pdi.patch_series_repr()'sería suficiente.

A veces, las distintas columnas deben tratarse de forma diferente cuando se agrupan. Por ejemplo, está perfectamente bien sumar sobre la cantidad, pero no tiene sentido sumar sobre el precio. El uso de '.agg' permite especificar distintas funciones de agregación para distintas columnas, como se muestra en la imagen:

<img src="Img/Articulo_2_diag_94.png">

O bien, puede crear varias funciones de agregación para una sola columna:

<img src="Img/Articulo_2_diag_95.png">

O bien, para evitar el engorroso cambio de nombre de columnas, puede hacer lo siguiente:

<img src="Img/Articulo_2_diag_96.png">

A veces, las funciones predefinidas no son lo suficientemente buenas para producir los resultados requeridos. Por ejemplo, sería mejor usar pesos al promediar el precio. Por lo tanto, puede proporcionar una función personalizada para eso. A diferencia de Series, la función puede acceder a varias columnas del grupo (se le proporciona un submarco de datos como argumento), como se muestra a continuación:

<img src="Img/Articulo_2_diag_97.png">

Lamentablemente, no se pueden combinar agregados predefinidos con funciones personalizadas de varias columnas, como la anterior, en un solo comando, ya que aggsolo acepta funciones de usuario de una sola columna. Lo único a lo que pueden acceder las funciones de usuario de una sola columna es el índice, que puede resultar útil en determinadas situaciones. Por ejemplo, ese día, se vendieron plátanos con un descuento del 50 %, como se puede ver a continuación:

<img src="Img/Articulo_2_diag_98.png">

Para acceder al valor del grupo por columna desde la función personalizada, se incluyó previamente en el índice.

Como es habitual, la función menos personalizada ofrece el mejor rendimiento. Por lo tanto, en orden de velocidad creciente:

- Función personalizada de varias columnas a través 'deg.apply()'

- Función personalizada de una sola columna a través de 'g.agg()' (admite aceleración con Cython o Numba)

- funciones predefinidas (objeto de función 'Pandas' o 'NumPy', o su nombre como cadena).

Una herramienta útil para mirar los datos desde una perspectiva diferente, a menudo utilizada junto con la agrupación, son las tablas dinámicas.


In [101]:
# Ejemplo de 'groupby'Archivos
data= {'client':['John','Silvia','Andrew'],
     'product':['bananas','oranges','bananas'],
     'quantity':[5, 3, 4],
     'price':[2, 5, 3],
     'total':[10, 15, 12]
}

df= pd.DataFrame(data)

print(df)
print()
print("Sumando por productos:")
print(df.groupby('product')[['quantity','total']].sum())
print()
print("Usando el metodo 'agg' para calcular 'suma', 'max' y 'mean'")
print(df.groupby('product').agg({'quantity':'sum', 'price':['max', 'min', 'mean']}))


   client  product  quantity  price  total
0    John  bananas         5      2     10
1  Silvia  oranges         3      5     15
2  Andrew  bananas         4      3     12

Sumando por productos:
         quantity  total
product                 
bananas         9     22
oranges         3     15

Usando el metodo 'agg' para calcular 'suma', 'max' y 'mean'
        quantity price         
             sum   max min mean
product                        
bananas        9     3   2  2.5
oranges        3     5   5  5.0


#### Pivotar y despivotar

Supongamos que tiene una variable aque depende de dos parámetros 'i' y 'j'. Hay dos formas equivalentes de representarla como tabla:

<img src="Img/Articulo_2_diag_99.png">

El formato "ancho" (wide format) es más apropiado cuando los datos son "densos" (cuando hay pocos elementos cero o faltantes), y el "largo" (long format) es mejor cuando los datos son "dispersos" (la mayoría de los elementos son ceros o faltantes y se pueden omitir de la tabla). La situación se complica cuando hay más de dos parámetros.

Naturalmente, debería haber una forma sencilla de realizar la conversión entre esos formatos, y Pandas ofrece una solución sencilla y cómoda para ello: la tabla dinámica.

Como ejemplo menos abstracto, considere la siguiente tabla con los datos de ventas. Dos clientes han comprado la cantidad designada de dos tipos de productos. Inicialmente, estos datos están en "formato largo". Para convertirlos al "formato ancho", utilice: 'df.pivot'

<img src="Img/Articulo_2_diag_100.png">

Este comando descarta todo lo que no esté relacionado con la operación (es decir, columnas de índice y precio) y transforma la información de las tres columnas solicitadas al formato largo, colocando los nombres de los clientes en el índice del resultado, los títulos de los productos en sus columnas y la cantidad vendida en el "cuerpo" del mismo.

En cuanto a la operación inversa, puedes utilizar stack. Combina 'index' y 'columns' en 'MultiIndex':

<img src="Img/Articulo_2_diag_101.png">

Si solo desea 'stack' determinadas columnas, puede utilizar 'melt':

<img src="Img/Articulo_2_diag_102.png">

Tenga en cuenta que meltordena las filas del resultado de manera diferente.

'pivot' pierde la información sobre el nombre del 'cuerpo' del resultado, por lo que con ambos 'stack' tenemos 'melt' que 'recordarle' a Pandas el nombre de la columna 'cantidad'.

En el ejemplo anterior, todos los valores están presentes, pero no es obligatorio:

<img src="Img/Articulo_2_diag_103.png">

La práctica de agrupar valores y luego pivotear los resultados es tan común que 'groupby' se pivothan agrupado en una función dedicada (y un método 'DataFrame' correspondiente) 'pivot_table':

- Sin el argumento 'columns', se comporta de manera similar a 'groupby';

- cuando no hay filas duplicadas para agrupar, funciona igual que pivot;

- De lo contrario, realiza agrupación y pivoteo.

<img src="Img/Articulo_2_diag_104.png">

El parámetro 'aggfunc' controla qué función de agregación se debe utilizar para agrupar las filas ( meanpor defecto).

Para mayor comodidad, 'pivot_table' puede calcular los subtotales y el total general:

<img src="Img/Articulo_2_diag_105.png">

Una vez creada, una tabla dinámica se convierte en un simple 'DataFrame' normal, por lo que se puede consultar utilizando los métodos estándar descritos anteriormente:

<img src="Img/Articulo_2_diag_106.png">

La mejor manera de entenderlo 'pivot_table' (¡además de empezar a usarlo de inmediato!) es seguir un estudio de caso relevante. Recomiendo encarecidamente dos de ellos:

- En esta entrada del blog se describe un caso de venta extremadamente completo.

- Un caso de uso genérico muy bien escrito (basado en el infame conjunto de datos del Titanic)

Las tablas dinámicas son especialmente útiles cuando se utilizan con 'MultiIndex'. Hemos visto muchos ejemplos en los que las funciones de 'Pandas' devuelven un 'DataFrame' con múltiples índices. Veámoslo con más detalle.


In [108]:
# PIVOTEANDO EL DATAFRAME
data= {'client':['John','Silvia','Andrew'],
     'product':['bananas','oranges','bananas'],
     'quantity':[5, 3, 4],
     'price':[2, 5, 3],
     'total':[10, 15, 12]
}
df= pd.DataFrame(data)
print("DataFrame Original") 
print(df)
print()
print("Aplicando el método 'stack'")
print(df.stack())
print()
 

DataFrame Original
   client  product  quantity  price  total
0    John  bananas         5      2     10
1  Silvia  oranges         3      5     15
2  Andrew  bananas         4      3     12

Aplicando el método 'stack'
0  client         John
   product     bananas
   quantity          5
   price             2
   total            10
1  client       Silvia
   product     oranges
   quantity          3
   price             5
   total            15
2  client       Andrew
   product     bananas
   quantity          4
   price             3
   total            12
dtype: object



In [115]:
# USANDO PIVOT
print(df)
print()
print("Creando con el método 'pivot' una tabla dinámica")
print("usando como índice: 'client', las columnas: 'product' y values: 'quantity'")
print(df.pivot(index='client', columns='product', values='quantity'))
print()
print("Usando 'melt' para reorganizar el DataFrame")
df_melted = df.melt(id_vars=['client', 'product'], value_vars=['quantity', 'price', 'total'], var_name='measurement', value_name='value')
print(df_melted)



   client  product  quantity  price  total
0    John  bananas         5      2     10
1  Silvia  oranges         3      5     15
2  Andrew  bananas         4      3     12

Creando con el método 'pivot' una tabla dinámica
usando como índice: 'client', las columnas: 'product' y values: 'quantity'
product  bananas  oranges
client                   
Andrew       4.0      NaN
John         5.0      NaN
Silvia       NaN      3.0

Usando 'melt' para reorganizar el DataFrame
   client  product measurement  value
0    John  bananas    quantity      5
1  Silvia  oranges    quantity      3
2  Andrew  bananas    quantity      4
3    John  bananas       price      2
4  Silvia  oranges       price      5
5  Andrew  bananas       price      3
6    John  bananas       total     10
7  Silvia  oranges       total     15
8  Andrew  bananas       total     12


## Parte 4. Multiíndice

<img src="Img/Articulo_2_diag_107.png">

El uso más sencillo de 'MultiIndex' para las personas que nunca han oído hablar de Pandas es utilizar una segunda columna de índice como complemento de la primera para identificar cada fila de forma única. Por ejemplo, para desambiguar ciudades de diferentes estados, el nombre del estado suele añadirse al nombre de la ciudad. (¿Sabías que hay alrededor de 40 Springfields en los EE. UU.?) En las bases de datos relacionales, se denomina clave principal compuesta.

Puede especificar las columnas que se incluirán en el índice después de analizar el 'DataFrame' desde 'CSV' o inmediatamente como argumento de 'read_csv'.

<img src="Img/Articulo_2_diag_108.png">

También puedes agregar niveles existentes al 'MultiIndex' posteriormente usando 'append=True', como puedes ver en la imagen a continuación:

<img src="Img/Articulo_2_diag_109.png">

Otro caso de uso, más típico de 'Pandas', es la representación de múltiples dimensiones en una situación en la que se dispone de una serie de objetos con un determinado conjunto de propiedades, especialmente cuando evolucionan con el tiempo. Por ejemplo:

- resultados de una encuesta sociológica,

- El conjunto de datos del 'Titanic',

- observaciones meteorológicas históricas,

- Una cronología de la clasificación del campeonato.

Esto también se conoce como 'Panel data' y Pandas debe su nombre a ello.

Agreguemos tal dimensión:

<img src="Img/Articulo_2_diag_110.png">

Ahora tenemos un espacio de cuatro dimensiones, donde

- Los años forman una dimensión (casi continua),

- Los nombres de las ciudades se colocan a lo largo del segundo,

- nombres de estados a lo largo del tercero, y

- Las propiedades particulares de la ciudad ('población', 'densidad', 'área', etc.) actúan como 'marcas de verificación' a lo largo de la cuarta dimensión.

El siguiente diagrama ilustra el concepto:

<img src="Img/Articulo_2_diag_111.png">

Para dejar espacio para los nombres de las dimensiones correspondientes a las columnas, Pandas desplaza todo el encabezado hacia arriba:

<img src="Img/Articulo_2_diag_112.png">


In [30]:
import numpy as np
import pandas as pd

df= pd.read_csv("Archivos/cities-states.csv")
print(df)
print()
print("Indice= 'city'")
df= pd.read_csv("Archivos/cities-states.csv", index_col=('city'))
print(df)
df= pd.read_csv("Archivos/cities-states.csv", index_col=('city','state'))
print()
print("Indice= 'city' y 'state'")
print(df)



          city     state  population   area
0     Portland    Oregon      609456  345.7
1  Springfield  Illinois      117006  158.4
2     Portland     Maine       66318   55.8
3  Springfield    Oregon       60177   41.1

Indice= 'city'
                state  population   area
city                                    
Portland       Oregon      609456  345.7
Springfield  Illinois      117006  158.4
Portland        Maine       66318   55.8
Springfield    Oregon       60177   41.1

Indice= 'city' y 'state'
                      population   area
city        state                      
Portland    Oregon        609456  345.7
Springfield Illinois      117006  158.4
Portland    Maine          66318   55.8
Springfield Oregon         60177   41.1


In [40]:
df1= pd.read_csv("Archivos/stats2010.csv", index_col=('city','state'))
df2= pd.read_csv("Archivos/stats2020.csv", index_col=('city','state'))

print(df1)
print()
print(df2)
print()

df3= pd.concat([df1,df2], axis= 1, keys= [2010, 2020])

print(df3)

print()

df4= df3.rename_axis(['year','property'], axis= 1)

print(df4)




                      population  density
city        state                        
Portland    Oregon        583776   1688.7
Springfield Illinois      116250    733.9
Portland    Maine          66194   1186.3
Springfield Oregon         59403   1445.3

                      population  density
city        state                        
Portland    Oregon        652503   1887.5
Springfield Illinois      114394    722.2
Portland    Maine          68408   1225.9
Springfield Oregon         61851   1504.9

                           2010               2020        
                     population density population density
city        state                                         
Portland    Oregon       583776  1688.7     652503  1887.5
Springfield Illinois     116250   733.9     114394   722.2
Portland    Maine         66194  1186.3      68408  1225.9
Springfield Oregon        59403  1445.3      61851  1504.9

year                       2010               2020        
property             

#### Agrupamiento aparente

Lo primero que hay que tener en cuenta sobre MultiIndex es que no agrupa nada, como podría parecer. Internamente, es solo una secuencia plana de etiquetas, como se puede ver a continuación:


In [41]:
df4.columns

MultiIndex([(2010, 'population'),
            (2010,    'density'),
            (2020, 'population'),
            (2020,    'density')],
           names=['year', 'property'])

Puedes obtener el mismo groupbyefecto para las etiquetas de fila simplemente ordenándolas:

In [43]:
df3.sort_index()

2010               2020        
                     population density population density
city        state                                         
Portland    Maine         66194  1186.3      68408  1225.9
            Oregon       583776  1688.7     652503  1887.5
Springfield Illinois     116250   733.9     114394   722.2
            Oregon        59403  1445.3      61851  1504.9

Incluso puedes desactivar la agrupación visual por completo configurando la opción de Pandas correspondiente : 'pd.options.display.multi_sparse=False'.

#### Conversiones de tipos

Pandas (así como el propio Python) hace una diferencia entre números y cadenas, por lo que suele ser una buena idea convertir números en cadenas en caso de que el tipo de datos no se haya detectado automáticamente:

<img src="Img/Articulo_2_diag_113.png">

Si te sientes aventurero, puedes hacer lo mismo con herramientas estándar:

<img src="Img/Articulo_2_diag_114.png">

Pero para usarlos correctamente, es necesario comprender qué son los "niveles" y los "códigos", mientras que 'pdi' permite trabajar con MultiIndex como si los niveles fueran listas ordinarias o matrices 'NumPy'.

Si realmente te lo preguntas, 'niveles' y 'códigos' son algo en lo que se divide una lista regular de etiquetas de un cierto nivel para acelerar operaciones como pivot, joiny así sucesivamente:

- pdi.get_level(df, 0) == Int64Index([2010, 2010, 2020, 2020])

- df.columns.levels[0] == Int64Index([2010, 2020])

- df.columns.codes[0] == Int64Index([0, 1, 0, 1])

#### Creación de un DataFrame con un MultiIndex

Además de leer archivos CSV y crear a partir de las columnas existentes, existen otros métodos para crear un MultiIndex. Se utilizan con menos frecuencia, principalmente para realizar pruebas y depurar errores.

La forma más intuitiva de utilizar la propia representación de MultiIndex de Panda no funciona por razones históricas.

In [2]:
import numpy as np
import pandas as pd

pd.MultiIndex([('2010','population'),
               ('2010','density'),
               ('2020','population'),
               ('2020','density')],
              names= ['year','property'])


TypeError: Must pass both levels and codes

Los 'niveles' y 'códigos' aquí se consideran (hoy en día) detalles de implementación que no deberían exponerse al usuario final, pero tenemos lo que tenemos.

Probablemente, la forma más sencilla de construir un MultiIndex es la siguiente:

In [28]:
import numpy as np
import pandas as pd

df= pd.DataFrame(
                 np.zeros((3, 4), int),
                 index= [['a','a','b'],
                         ['c','d','c']],
                 columns= [['A','A','B','B'],
                           ['C','D','C','D']])

print("df= pd.DataFrame(np.zeros((3, 4), int),\n\n        index= [['a','a','b'],\n                ['c','d','c']],\n\n      columns= [['A','A','B','B'],\n                ['C','D','C','D']])")
print()
print(df)
print()

df1= pd.DataFrame(np.zeros((3, 4), int))

df2= index= [['a','a','b'],
             ['c','d','c']],

columns= [['A','A','B','B'],
              ['C','D','C','D']]


print("df1= pd.DataFrame(np.zeros((3, 4), int))")
print(df1)
print()

df= pd.DataFrame(np.zeros((3, 4), int),

        index= [['a','a','b'],
                ['c','d','c']],

      columns= [['A','A','B','B'],
                ['C','D','C','D']])

     A     B   
     C  D  C  D
a c  0  0  0  0
  d  0  0  0  0
b c  0  0  0  0

df1= pd.DataFrame(np.zeros((3, 4), int))
   0  1  2  3
0  0  0  0  0
1  0  0  0  0
2  0  0  0  0



In [29]:
print("Metodo chaining")
df= df.rename_axis(['K','L'], axis=0).rename_axis(['M','N'], axis=1)
print(df)
print()
print("df.index.names = ['K','L']")
print("df.column.names = ['M','N']")

Metodo chaining
M    A     B   
N    C  D  C  D
K L            
a c  0  0  0  0
  d  0  0  0  0
b c  0  0  0  0

df.index.names = ['K','L']
df.column.names = ['M','N']


La desventaja es que los nombres de los niveles deben asignarse en una línea separada o en un método encadenado independiente. Varios constructores alternativos agrupan los nombres junto con las etiquetas.

In [38]:
print("MEDIANTE ARREGLOS")
mi= pd.MultiIndex.from_arrays([['a','b','c'], ['d','e','f']], names=['K','L'])
print(mi)
print()

df= pd.DataFrame([[10, 20],[30, 40],[50, 60]],
                 index= mi,
                 columns= ['A','B'])
print(df)
print()
print("MEDIANTE TUPLAS")
mi= pd.MultiIndex.from_tuples([('a','d'),('b','e'), ('c','f')], names=['K','L'])
print(mi)
print()
df1= pd.DataFrame([[10, 20],[30, 40],[50, 60]],
                 index= mi,
                 columns= ['A','B'])
print(df1)
print()
print("MEDIANTE DICCIONARIO")
"""
df2= pd.DataFrame({'A':[10, 20, 30], 'B':[40, 50, 60]},
                  index= pdi.from_dict({'K':['a','b','c'],
                                       'L':['d','e','f']}))

print(df2)

"""


MEDIANTE ARREGLOS
MultiIndex([('a', 'd'),
            ('b', 'e'),
            ('c', 'f')],
           names=['K', 'L'])

      A   B
K L        
a d  10  20
b e  30  40
c f  50  60

MEDIANTE TUPLAS
MultiIndex([('a', 'd'),
            ('b', 'e'),
            ('c', 'f')],
           names=['K', 'L'])

      A   B
K L        
a d  10  20
b e  30  40
c f  50  60

MEDIANTE DICCIONARIO


"\ndf2= pd.DataFrame({'A':[10, 20, 30], 'B':[40, 50, 60]},\n                  index= pdi.from_dict({'K':['a','b','c'],\n                                       'L':['d','e','f']}))\n\nprint(df2)\n\n"

Cuando los niveles forman una estructura regular, puedes especificar los elementos clave y dejar que Pandas los intercale automáticamente, como se muestra a continuación:

<img src="Img/Articulo_2_diag_115.png">

Todos los métodos enumerados anteriormente también se aplican a las columnas. Por ejemplo:

<img src="Img/Articulo_2_diag_116.png">

#### Indexación con MultiIndex

Lo bueno de acceder a 'DataFrame' a través de 'MultiIndex' es que puedes referenciar fácilmente todos los niveles a la vez (omitiendo potencialmente los niveles internos) con una sintaxis agradable y familiar.

Columnas — mediante corchetes regulares

<img src="Img/Articulo_2_diag_117.png">

Filas y celdas: uso.loc[]

<img src="Img/Articulo_2_diag_118.png">

Ahora bien, ¿qué sucede si desea seleccionar todas las ciudades de Oregón o dejar solo las columnas con población? La sintaxis de Python impone dos limitaciones:

1. No hay forma de distinguir entre ambos 'df['a', 'b']' y 'df[('a', 'b')]': se procesa de la misma manera, por lo que no puedes escribir simplemente 'df[:, 'Oregon']'. De lo contrario, Pandas nunca sabría si te refieres a Oregon la columna o Oregon el segundo nivel de filas.

2. Python solo permite dos puntos dentro de corchetes, no dentro de paréntesis, por lo que no puedes escribir 'df.loc[(:, 'Oregon'), :]'.

Desde el punto de vista técnico, no es difícil organizarlo. He parcheado el DataFrame (es decir, he creado un parche que se descarta cuando el núcleo muere) para agregar esa funcionalidad, que puedes ver aquí:

<img src="Img/Articulo_2_diag_119.png">

La única desventaja de esta sintaxis es que cuando se utilizan ambos indexadores, devuelve una copia, por lo que no se puede escribir 'df.mi[:,’Oregon’].co[‘population’] = 10'. Hay muchos indexadores alternativos, algunos de los cuales permiten este tipo de asignaciones, pero todos tienen sus propias peculiaridades:

1. Puedes intercambiar capas internas con capas externas y usar los corchetes.

<img src="Img/Articulo_2_diag_120.png">

Por lo tanto, 'df[:, ‘population’]' se puede implementar con
'df.swaplevel(axis=1)['population']'

Esto parece extraño y no es conveniente para más de dos niveles.

2. Puedes utilizar el método 'xs':
'df.xs(‘population’, level=1, axis=1)'.

No parece lo suficientemente Python, especialmente cuando se seleccionan varios niveles.
Este método no puede filtrar filas y columnas al mismo tiempo, por lo que el motivo del nombre 'xs'(que significa "sección transversal") no está del todo claro. No se puede utilizar para establecer valores.

3. El método preferido para manejar esta situación es crear un alias para pd.IndexSlicey usarlo dentro de '.loc':
'idx=pd.IndexSlice; df.loc[:, idx[:, 'population']]'

Eso es más Python, pero la necesidad de crear un alias para acceder a un elemento es algo complicado (y es demasiado largo sin un alias). Puedes seleccionar filas y columnas al mismo tiempo. Escribible.

4. Puedes aprender a utilizar sliceen lugar de dos puntos. Si lo sabes, 'a[3:10:2] == a[slice(3,10,2)]'  es posible que también entiendas lo siguiente: 'df.loc[:, (slice(None), 'population')]', pero de todos modos es difícil de leer. Puede seleccionar filas y columnas al mismo tiempo. Es escribible.

En resumen, Pandas tiene varias formas de acceder a elementos del DataFrame con MultiIndex usando corchetes, pero ninguna de ellas es lo suficientemente conveniente, por lo que tuvieron que adoptar una sintaxis de indexación alternativa:

5. Un mini-lenguaje para el .querymétodo (es el único que es capaz de hacer 'o', no sólo 'y'):
'df.query('state=="Oregon" or city=="Portland"')'.

Es conveniente y rápido, pero carece de soporte de IDE (no tiene autocompletado, resaltado de sintaxis, etc.) y solo filtra las filas, no las columnas. Eso significa que no se puede implementar 'df[:, 'population']' sin transponer el DataFrame (que perderá los tipos a menos que todas las columnas sean del mismo tipo). No se puede escribir.

A continuación se muestra una tabla resumen de todos los métodos de indexación MultiIndex:

<img src="Img/Articulo_2_diag_121.png">

Ninguno de ellos es perfecto, pero algunos se acercan.

#### Stacking and unstacking 

Pandas no tiene columnas 'set_index'. Una forma habitual de agregar niveles a las columnas es "desapilar" los niveles existentes del índice:

<img src="Img/Articulo_2_diag_122.png">

El sistema 'stack' de Pandas es muy diferente al 'stack'vde NumPy. Veamos qué dice la documentación sobre las convenciones de nombres:

“La función recibe su nombre por analogía con una colección de libros que se reorganizan desde estar uno al lado del otro en una posición horizontal (las columnas del marco de datos) a estar apilados verticalmente uno sobre el otro (en el índice del marco de datos)”.

La parte "on top" no me suena muy convincente, pero al menos esta explicación ayuda a memorizar cuál mueve las cosas en qué dirección. Por cierto, Series tiene 'unstack', pero no tiene 'stack' porque ya está "apilada". Al ser unidimensional, Series puede actuar como vector de fila o vector de columna en diferentes situaciones, pero normalmente se las considera vectores de columna (por ejemplo, columnas de marcos de datos).

Por ejemplo:

<img src="Img/Articulo_2_diag_123.png">

También puede especificar qué nivel apilar o desapilar por nombre o por índice posicional. En este ejemplo, df.stack(), df.stack(1)y df.stack(‘year’)producen el mismo resultado, así como df1.unstack(), df1.unstack(2), y df1.unstack(‘year’). El destino siempre es "después del último nivel" y no es configurable. Si necesita colocar el nivel en otro lugar, puede usar df.swaplevel().sort_index()opdi.swap_level(df, sort=True)

No debe 'columns' contener valores duplicados para ser elegible para apilar (lo mismo se aplica para indexdesapilar):

<img src="Img/Articulo_2_diag_124.png">

También puede especificar qué nivel apilar o desapilar por nombre o por índice posicional. En este ejemplo, 'df.stack()', 'df.stack(1)' y 'df.stack(‘year’)' producen el mismo resultado, así como 'df1.unstack()', 'df1.unstack(2)', y 'df1.unstack(‘year’)'. El destino siempre es "después del último nivel" y no es configurable. Si necesita colocar el nivel en otro lugar, puede usar 'df.swaplevel().sort_index()opdi.swap_level(df, sort=True)'

No debe 'columns' contener valores duplicados para ser elegible para 'apilar' (lo mismo se aplica para 'index' para 'desapilar'):

<img src="Img/Articulo_2_diag_126.png">

Se podría especular que si el lunes de 'John' está a la izquierda del viernes de 'John', entonces ‘Mon’ < ‘Fri’, y de manera similar, ‘Fri’ < ‘Sun’ para Silvia, por lo que el resultado debería ser ‘Mon’ < ‘Fri’ < ‘Sun’. Esto es legítimo, pero ¿qué pasa si las columnas restantes están en un orden diferente, digamos, ‘Mon’ < ‘Fri’y ‘Tue’ < ‘Fri'? O ‘Mon’ < ‘Fri’y ‘Wed’ < ‘Sat’?

Vale, no hay tantos días de la semana y Pandas podría deducir el orden basándose en el conocimiento previo. Pero la humanidad no ha llegado a una conclusión decisiva sobre si el domingo debería estar al final o al principio de la semana. ¿Qué orden debería utilizar Pandas por defecto? ¿Leer configuraciones regionales? ¿Y qué pasa con secuencias menos triviales, por ejemplo, el orden de los estados de los EE.UU.?

Lo que hace Pandas en esta situación es simplemente ordenarlo alfabéticamente, como puedes ver a continuación:

<img src="Img/Articulo_2_diag_127.png">

Aunque se trata de una opción predeterminada sensata, sigue pareciendo incorrecta. ¡Debería haber una solución! Y la hay. Se llama 'CategoricalIndex'. Recuerda el orden incluso si faltan algunas etiquetas. Recientemente se ha integrado sin problemas en la cadena de herramientas de Pandas. Lo único que le falta es infraestructura. Es difícil de construir; es frágil (recurre al tipo de datos de objeto en ciertas operaciones), pero es perfectamente utilizable y la biblioteca 'pdi' tiene algunos ayudantes para acentuar la curva de aprendizaje.

Por ejemplo, para indicarle a Pandas que bloquee el orden de, digamos, un índice simple que contiene los productos (que inevitablemente se ordenarán si decides desapilar los días de la semana en columnas), necesitas escribir algo tan horrendo como 'df.index = pd.CategoricalIndex(df.index, df.index, sorted=True)'. Y es mucho más artificial para 'MultiIndex'.

La biblioteca 'pdi' tiene una función auxiliar 'locked' (y un alias 'lock' que tiene 'inplace=True' por defecto) para bloquear el orden de un determinado nivel de MultiIndex promoviendo el nivel a 'CategoricalIndex':

<img src="Img/Articulo_2_diag_128.png">

La marca de verificación '✓' junto al nombre de un nivel significa que el nivel está bloqueado. Se puede visualizar de forma manual con 'pdi.vis(df)' o de forma automática aplicando un parche a la representación HTML del marco de fecha con 'pdi.vis_patch()'. Después de aplicar el parche, simplemente escribiendo "df" en una celda de Jupyter se mostrarán las marcas de verificación de todos los niveles con ordenamiento bloqueado.

'lock' y 'locked' funcionan automáticamente en casos simples (como nombres de clientes), pero necesitan una pista del usuario para los casos más complejos (como días de la semana con días faltantes).

<img src="Img/Articulo_2_diag_129.png">

Una vez que se cambia el nivel a ''CategoricalIndex', mantiene el orden original en operaciones como 'sort_index', 'stack', 'unstack', 'pivot, 'pivot_table', etc.

Sin embargo, es frágil. Incluso una operación tan inocente como agregar una columna a través de 'df[‘new_col’] = 1' la misma la rompe. Utilice 'pdi.insert(df.columns, 0, ‘new_col’, 1)' qué nivel(es) de procesos son los 'CategoricalIndex' correctos.

#### Manipulando niveles (levels)

Además de los métodos ya mencionados, existen algunos más:

- 'pdi.get_level' (obj, level_id) devuelve un nivel particular referenciado por número o por nombre, funciona con 'DataFrames', 'Series' y 'MultiIndex', un alias para 'df.columns.get_level_values';

- 'pdi.set_level' (obj, level_id, labels) reemplaza las etiquetas de un nivel con la matriz dada ('lista', 'arrays', 'NumPy', 'Serie', 'Índice', etc.), — no tiene equivalente directo en  puro Pandas:

<img src="Img/Articulo_2_diag_130.png">

- 'pdi.insert_level' (obj, pos, labels, name) agrega un nivel con los valores dados (transmitidos correctamente si es necesario), — no se puede hacer fácilmente en puro Pandas;

- 'pdi.drop_level' (obj, level_id) elimina el nivel especificado del 'MultiIndex' (agrega inplaceargumento a 'df.droplevel'):

<img src="Img/Articulo_2_diag_131.png">

- 'pdi.swap_levels' (obj, src=-2, dst=-1) intercambia dos niveles (dos niveles más internos por defecto), agrega argumentos 'inplace' y 'sort' a 'df.swaplevel'

- 'pdi.move_level'  (obj, src, dst) mueve un nivel particular srca la posición designada 'dst'(no se puede hacer fácilmente en puro Pandas):

<img src="Img/Articulo_2_diag_132.png">

Además de los argumentos mencionados anteriormente, todas las funciones de esta sección tienen los siguientes argumentos:

- 'axis=None' donde 'None' significa 'columnas' para un DataFrame e 'índice' para una Serie (también conocido como eje 'información');

- 'sort=False', opcionalmente ordena el 'MultiIndex' correspondiente después de las manipulaciones;

- 'inplace=False', opcionalmente realiza la manipulación en el lugar (no funciona con un solo Indexporque es inmutable).

Todas las operaciones anteriores entienden la palabra nivel en el sentido convencional ( nivel tiene el mismo número de etiquetas que el número de columnas en el DataFrame), ocultando la maquinaria de 'index'.label' y 'index.codes' al usuario final.

En las raras ocasiones en que mover e intercambiar niveles separados no es suficiente, puedes reordenar todos los niveles a la vez con esta llamada pura de Pandas:

'df.columns = df.columns.reorder_levels(['M','L','K'])'
donde ['M', 'L', 'K'] es el orden deseado de los niveles.

Generalmente, es suficiente usar 'get_level' y ''set_level' hacer las correcciones necesarias a las etiquetas, pero si desea aplicar una transformación a todos los niveles del 'MultiIndex' a la vez, Pandas tiene una función (con nombre ambiguo) 'rename' que acepta un 'dict' o una función:

<img src="Img/Articulo_2_diag_133.png">

En cuanto al cambio de nombre de los niveles, sus nombres se almacenan en el campo '.names'. Este campo no admite asignaciones directas (¿por qué no?): pero se puede reemplazar como un todo: Alternativamente, puede utilizar un : encadenable

'df.index.names[1] = ‘x’ ' # TypeError

'df.index.names = ['z', 'x']' # ok

rename_axis

<img src="Img/Articulo_2_diag_134.png">

Cuando solo necesitas cambiar el nombre de un nivel en particular, la sintaxis es la siguiente:

<img src="Img/Articulo_2_diag_135.png">

O si desea hacer referencia al nivel por número en lugar de por nombre, utilice o (ambos también pueden funcionar por nombre).

'df.index = df.index.set_names(‘z’, level=0)'

'pdi.rename_level(df, 'z', 0, axis=0)'

#### Convertir MultiIndex en un índice plano y restaurarlo

Como hemos visto anteriormente, el método 'query' conveniente solo resuelve la complejidad de trabajar con 'MultiIndex' en las filas. Y a pesar de todas las funciones auxiliares, cuando alguna función complicada de Pandas devuelve un 'MultiIndex' en las columnas, tiene un efecto de 'shock' para los principiantes. Por lo tanto, la biblioteca pdi tiene lo siguiente:

- 'join_levels (obj, sep=’_’, name=None) Une todos los niveles de MultiIndex en un solo índice

- 'split_level' (obj, sep=’_’, names=None)Divide el índice nuevamente en un Multiíndice

<img src="Img/Articulo_2_diag_136.png">

Ambos tienen argumentos opcionales 'axis' y  'inplace'.

En cuanto a una solución pura Pandas el siguiente código puede funcionar:

- niveles de unión:

'df.columns = ['_'.join(k) for k in df.columns.to_flat_index()]'

- niveles divididos:

'df.columns = pd.MultiIndex.from_tuples(k.split('_') for k in df.columns)'

#### Ordenación de MultiIndex

Dado que MultiIndex consta de varios niveles, la ordenación es un poco más complicada que para un único índice. Se puede hacer con el sort_indexmétodo, pero se puede ajustar aún más con los siguientes argumentos:

<img src="Img/Articulo_2_diag_137.png">

Para ordenar los niveles de columna, especifique 'axis=1'.

#### Lectura y escritura de marcos de datos multiindexados en el disco

Pandas puede escribir un 'DataFrame' con un 'MultiIndex' en un archivo CSV de forma totalmente automática: 'df.to_csv('df.csv’)'. Sin embargo, al leer un archivo de este tipo, Pandas no puede analizar el 'MultiIndex' automáticamente y necesita algunas sugerencias del usuario. Por ejemplo, para leer un ''DataFrame' con columnas de tres niveles de alto y un índice de cuatro niveles de ancho, debe especificar

'pd.read_csv('df.csv', header=[0,1,2], index_col=[0,1,2,3])'.

Esto significa que las primeras tres líneas contienen la información sobre las columnas, y los primeros cuatro campos en cada una de las líneas subsiguientes contienen los niveles de índice (si hay más de un nivel en columns, no puede hacer referencia a los niveles de fila por nombres en read_csv, solo por números).

No es conveniente descifrar manualmente el número de niveles en la columna 'MultiIndex', por lo que una mejor idea sería descifrar 'stack()' todos los niveles del encabezado de columna menos uno antes de guardar el 'DataFrame' en CSV, y 'unstack()' volver a descifrarlos después de leerlos.

Si necesita una solución que se ejecute y se olvide, es posible que desee considerar los formatos binarios, como el formato 'pickle' de 'Python':

- directamente: 'df.to_pickle('df.pkl'), pd.read_pickle('df.pkl')'

- usando 'storemagic' en 'Jupyter' '%store df' entonces (almacena en) '%store -r df'

'$HOME/.ipython/profile_default/db/autorestore'

Este formato es pequeño y rápido, pero solo se puede acceder a él desde Python. Si necesita interoperabilidad con otros ecosistemas, busque formatos más estándar, como el formato Excel (que requiere las mismas sugerencias que 'read_csv' para leer 'MultiIndex'). Aquí está el código:

!pip install openpyxl 
df.to_excel( 'df.xlsx' ) 
df1 = pd.read_excel( 'df.xlsx' , encabezado=[ 0 , 1 , 2 ], índice_col=[ 0 , 1 , 2 , 3 ])

El formato de archivo 'Parquet' admite DataFrame multiindexados sin ningún tipo de sugerencia (la única limitación es que todas las etiquetas de columna deben ser cadenas), produce archivos más pequeños y funciona más rápido (consulte un punto de referencia):

df.to_parquet( 'df.parquet' ) 
df1 = pd.read_parquet( 'df.parquet' )

La documentación oficial de 'Pandas' tiene una tabla que enumera los aproximadamente 20 formatos compatibles.

#### Aritmética de MultiIndex

En las operaciones en las que se utiliza un DatFrame multiindexado como un todo, se aplican las mismas reglas que para los DataFrame comunes (consulte la Parte 3). Sin embargo, trabajar con un subconjunto de celdas tiene algunas peculiaridades propias.

Puede actualizar un subconjunto de columnas referenciadas a través de los niveles MultiIndex externos de manera tan sencilla como la siguiente:

<img src="Img/Articulo_2_diag_138.png">

O si desea mantener intactos los datos originales,.

'df1 = df.assign(population=df.population*10)'

También puedes obtener fácilmente la densidad de población con

'density=df.population/df.area'.

<img src="Img/Articulo_2_diag_139.png">

Pero desafortunadamente, no puedes asignar el resultado al marco de datos original con 'df.assign'.

Un enfoque es apilar todos los niveles irrelevantes del índice de columna en el índice de fila, realizar los cálculos necesarios y volver a desapilarlos (usarlo pdi.lockpara mantener el orden original de las columnas).

<img src="Img/Articulo_2_diag_140.png">

Alternativamente, puedes utilizar 'pdi.assign':

<img src="Img/Articulo_2_diag_141.png">

'pdi.assign' es consciente del orden de bloqueo, por lo que si le suministra un 'DataFrame' con niveles bloqueados, no los desbloqueará para que las operaciones de apilado/desapilado/etc. posteriores mantengan las columnas y filas originales en orden.

Un excelente ejemplo de procesamiento de un conjunto de datos de ventas de la vida real con un MultiIndex enorme se puede encontrar aquí.

En definitiva, 'Pandas' es una gran herramienta para analizar y procesar datos. Esperamos que este artículo te haya ayudado a entender tanto el "cómo" como el "por qué" de la solución de problemas típicos, y a apreciar el verdadero valor y la belleza de la biblioteca Pandas.